In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import os
os.chdir('/home/maksym/advbench')
# TODO: a temporary hack. Later on rather use `pip install`.



First install **`AdvBench`**:



In [3]:
# !pip install -q git+https://github.com/fra31/advbench
# !pip install -q -r torch==1.4.0 requests 
# TODO: [later] install should be done with pip ideally



Now let's try to restore the most robust model from [Carmon2019Unlabeled](https://arxiv.org/abs/1905.13736) that 
achieves 59.50% evaluated with AA+:



In [9]:
from data import load_cifar10
from utils import load_model, clean_accuracy

x_test, y_test = load_cifar10(n_examples=50)
model = load_model(model_name='Carmon2019Unlabeled').cuda().eval()

acc = clean_accuracy(model=model, x=x_test, y=y_test)
print('Clean accuracy: {:.2%}'.format(acc))



Files already downloaded and verified
Clean accuracy: 92.00%


92% clean accuracy is quite reasonable which means that we have restored the model successfully!

Now let's try to evaluate its robustness with a cheap version [AutoAttack](https://arxiv.org/abs/2003.01690) from 
ICML'20 with 2/4 attacks (only A-PGD-CE and A-PGD-DLR):



In [13]:
from attacks.autoattack import AutoAttack
adversary = AutoAttack(model, norm='Linf', eps=8/255, plus=False, attacks_to_run=['apgd-ce', 'apgd-dlr'])
adversary.cheap()
x_adv = adversary.run_standard_evaluation(x_test, y_test)



initial accuracy: 92.00%
apgd-ce - 1/1 - 19 out of 46 successfully perturbed
robust accuracy after APGD-CE: 54.00% (total time 10.3 s)
apgd-dlr - 1/1 - 1 out of 27 successfully perturbed
robust accuracy after APGD-DLR: 52.00% (total time 17.0 s)
max Linf perturbation: 0.03137, nan in tensor: 0, max: 1.00000, min: 0.00000
robust accuracy: 52.00%


Note that for our standardized evaluation of Linf-robustness we use the *full* version of AutoAttack which is slower but 
more accurate (for that just use `adversary = AutoAttack(model, norm='Linf', eps=8/255, plus=False)`).

You can also easily plug in any existing library with adversarial attacks such as [FoolBox](https://github.com/bethgelab/foolbox):



In [14]:
# !pip install -q foolbox 



You are using pip version 18.1, however version 20.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [15]:
import foolbox as fb
fmodel = fb.PyTorchModel(model, bounds=(0, 1))

_, advs, success = fb.attacks.LinfPGD()(fmodel, images, labels, epsilons=[8/255])

SyntaxError: invalid syntax (tensor.py, line 544)

In [ ]:

### DEBUG


